## 0. Database config

In [1]:
DB_NAME = 'football'
SCHEMA_PATH = '../assets/FootballDatabase.schema.sql'
DATA_PATH = '../data/'

## 1. Required dependencies

In [2]:
!pip install mysql-connector-python python-dotenv

## 2. Imports and config loading

In [3]:
import os
from dotenv import load_dotenv

import mysql.connector
from mysql.connector import errorcode

# Load config values from .env file
load_dotenv()

DB_CONFIG = {
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASS'),
    'host': os.getenv('DB_HOST'),
    'allow_local_infile': True,
    'raise_on_warnings': True
}

## 3. Auxiliary functions

### 3.1. Mysql connection

In [4]:
def mysql_connect(config):

    try:
        cnx = mysql.connector.connect(**config)
        return cnx

    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print(f'[CONNERROR] Invalid credentials: {err}')
            
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print(f'[CONNERROR] Database does not exist: {err}')
            
        else:
            print(f'[CONNERROR] {err}')
            
        return None

### 3.2. Mysql close connection

In [5]:
def mysql_close_connection(cnx):
    
    try:
        cnx.close()
        
    except mysql.connector.Error as err:
        print(f'[CLOSEERROR] Failed closing connection to mysql: {err}')
        raise

### 3.3. Database creation

In [6]:
def create_database(cnx, dbname, charset='UTF8MB4'):
    
    try:
        cursor = cnx.cursor()
        cursor.execute('DROP DATABASE IF EXISTS %s', (dbname))
        cursor.execute('CREATE DATABASE %s DEFAULT CHARACTER SET "%s"', (dbname, charset))
        
    except mysql.connector.Error as err:
        print(f'[CREATEERROR] Failed creating database {dbname}: {err}')
        raise

### 3.4. Tables creation

In [7]:
import re

def tables_from_schema(schema_path):
    
    with open(schema_path, 'r') as schema_file:
        schema = schema_file.read()
    
        # Split each `CREATE TABLE ...` in schema as an element of create_table array
        create_table = re.split('(?<!^)\s+(?=(CREATE))', schema)[::2]
    
        # Creata a dictionary with table name as key and create table script as value
        # ==> table['league'] = 'CREATE TABLE `league` ( ...'
        tables = {re.search(r'(?<=`)\w+', table)[0]: table for table in create_table}
    
        schema_file.close()
        return tables

In [8]:
def create_tables(cnx, dbname, schema):
    
    try:
        cursor = cnx.cursor()
        cursor.execute(f'USE {dbname}')
        print(f'Using database {dbname}')
        
    except mysql.connector.Error as err:
        print(f'Database {dbname} does not exists')

        if err.errno == errorcode.ER_BAD_DB_ERROR:
            create_database(cnx, dbname)
            print(f'Database {dbname} created successfully')
            cnx.database = dbname

        else:
            print(f'[TABLEERROR] Failed creating database: {err}')
            raise
    
    tables = tables_from_schema(schema)
    
    for table_name, table_script in tables.items():
        try:
            print(f'Creating table {table_name}: ', end='')
            cursor.execute('SET FOREIGN_KEY_CHECKS=0')
            cursor.execute(f'DROP TABLE IF EXISTS {table_name}')
            cursor.execute('SET FOREIGN_KEY_CHECKS=1')
            cursor.execute(table_script)
            
        except mysql.connector.Error as err:
            print(f'[TABLEERROR] Failed creating table {table_name}: {err}')
            raise
            
        else:
            print('OK')
            
    return tables

### 3.5. Data load

In [9]:
def load_data(cnx, dbname, tables, data_folder):
    
    cursor = cnx.cursor()
    cursor.execute(f'USE {dbname}')

    for table_name in tables:
        try:
            print(f'Loading data in table {table_name}: ', end='')
            infile_path = os.path.realpath(f'{data_folder}/{table_name}s.csv')
            outfile_path = os.path.realpath(f'{data_folder}/{table_name}s_out.csv')
            
            with open(infile_path, 'r') as infile, open(outfile_path, 'w') as outfile:
                data = infile.read().replace('NA', r'\N')
                outfile.write(data)
                
            cursor.execute(f'''
                LOAD DATA LOCAL INFILE '{outfile_path}'
                INTO TABLE {table_name}
                FIELDS TERMINATED BY ',' 
                ENCLOSED BY '"'
                LINES TERMINATED BY '\n'
                IGNORE 1 ROWS;'''
            )
            cnx.commit()
            os.remove(outfile_path)
            
        except mysql.connector.Error as err:
            print(f'[LOADERROR] Failed loading data on table {table_name}: {err}')
            raise
            
        else:
            print('OK')

## 4. Executing

In [10]:
%%time

cnx = mysql_connect(DB_CONFIG)

%time tables = create_tables(cnx, DB_NAME, SCHEMA_PATH)
%time load_data(cnx, DB_NAME, tables, DATA_PATH)

mysql_close_connection(cnx)

Using database football
Creating table player: OK
Creating table team: OK
Creating table league: OK
Creating table game: OK
Creating table appearance: OK
Creating table teamstat: OK
Creating table shot: OK
CPU times: user 139 ms, sys: 16 ms, total: 155 ms
Wall time: 7.65 s
Loading data in table player: OK
Loading data in table team: OK
Loading data in table league: OK
Loading data in table game: OK
Loading data in table appearance: OK
Loading data in table teamstat: OK
Loading data in table shot: OK
CPU times: user 3.88 s, sys: 1 s, total: 4.88 s
Wall time: 38.2 s
CPU times: user 4.23 s, sys: 1.02 s, total: 5.25 s
Wall time: 47.4 s
